In [60]:
import pandas as pd
import numpy as np

df = pd.read_excel('node-edges.xlsx', header=None, )
df.columns = df.iloc[61]
df = df.iloc[0:30]

count = 0
for i in range(0, len(df.columns)):
    if (i % 2 != 0):
        df.columns.values[i] = ("%02d" % count) + ': Destination'
        count += 1
        
slices = []
for i in range(2, len(df.columns), 2):
    s = df.iloc[:, i-2:i]
    s = s.dropna()
    s = s[s.iloc[:,1] != '-'] # edges are marked at entrance AND destination, so missing values can be removed
   
    # Edges without a destination node have to be handled in the algo. 
    # Eg. Marts doesnt generate any edges in the google sheet.
    # # Remove unpassables. This messes up the path finding algorithm because it can't retrieve a target node for them.
    # unpassables = ['One-Way', 'Dead-End', 'Event', 'Trainer', 'Strength', 'Rock Smash', 'Surf', 'Waterfall', 'Dive', 'Custom 1', 'Custom 2', 'Custom 3']
    # # the ~ negates the boolean index
    # s = s[~s.iloc[:,1].isin(unpassables)]
    
    slices.append(s)
    
nodes = {}
edges = {}
for s in slices:
    node_name = s.iloc[:,0].name
    node_name = str.strip(node_name.split(':')[1]) # remove number and whitespace from name
    edges.update({e: node_name for e in s.iloc[:,0]})
    # TODO: algorithm may need incoming edges, so the list may need to be packaged in a dict
    # nodes.update({node_name: [{'entrance': e, 'destination': d} for e, d in s.iloc]})
    # nodes.update({node_name: {'transitions': {e: d for e, d in s.iloc}}})
    nodes.update({node_name: {'node_name': node_name,
                              'transitions': [{'entrance': e, 'destination': d} for e, d in s.iloc],
                              'visited':  False,
                              'previous': []}})
    
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        
graph = Graph(nodes, edges)

In [75]:
import copy

def graph_algo(src, dest, graph):
    if src == dest:
        return True
    src_node = graph.nodes[src]
    src_node['visited'] = True
    queue = [src_node]
    # counter = 0
    # while len(queue) > 0 and counter <= 2:
    while len(queue) > 0:
        # print('Queue: ', sorted(list(map(lambda x: x['node_name'], queue))))
        print('Queue: ', sorted([x['node_name'] for x in queue]))
        # print('##### Queue: ', queue)
        # print('##### Queue #####')
        # for n in sorted(queue, key=lambda x: x['node_name']):
        #     print(n['node_name'])
        #     print(n['visited'])
        current = queue.pop(0)
        # current['visited'] = True
        # if current['visited'] == True:
        #     continue
        if current['node_name'] == dest:
            print('Current: %s equals Dest: %s' % (current['node_name'], dest))
            return True
        # queue.append([t['destination']] for t in current_node['transitions'])
        # print('Processing Transitions...')
        for t in current['transitions']:
            node_name = graph.edges.get(t['destination'])
            # print('Node Name: ', node_name)
            if node_name:
                node = graph.nodes[node_name]
                # print('Node: ', node)
                if not node['visited']:
                    node['visited'] = True
                    queue.append(node)
        # counter += 1
            # queue.append(graph.edges[t['destination']])
    return False

graph_algo('Slateport/Mauville/Verdanturf', 'Rustboro', copy.deepcopy(graph))

Queue:  ['Slateport/Mauville/Verdanturf']
Queue:  ['Abandoned Ship', 'Aqua Hideout', 'E4 Pokecenter', 'Fallarbor/Lavaridge', 'Granite Cave', 'Magma Hideout', 'Meteor Falls', 'Mossdeep', 'Mt Pyre', 'Oldale/Petalburg', 'Places of Interest', 'Pokecenters']
Queue:  ['Abandoned Ship', 'Dewford', 'E4 Pokecenter', 'Fallarbor/Lavaridge', 'Fortree/Lilycove', 'Granite Cave', 'Lilycove Contest Hall', 'Lilycove Department Store', 'Magma Hideout', 'Meteor Falls', 'Mossdeep', 'Mt Pyre', 'Oldale/Petalburg', 'Places of Interest', 'Pokecenters', 'Route 119/123']
Queue:  ['Abandoned Ship', 'Dewford', 'E4 Pokecenter', 'Fallarbor/Lavaridge', 'Fortree/Lilycove', 'Granite Cave', 'Lilycove Contest Hall', 'Lilycove Department Store', 'Magma Hideout', 'Meteor Falls', 'Mossdeep', 'Mt Pyre', 'Oldale/Petalburg', 'Places of Interest', 'Route 119/123', 'Rustboro', 'Sootopolis', 'Victory Road']
Queue:  ['Abandoned Ship', 'Dewford', 'E4 Pokecenter', 'Fallarbor/Lavaridge', 'Fortree/Lilycove', 'Granite Cave', 'Lilycove

True

In [35]:
list(edges.keys())
len(list(nodes.keys()))
sorted(list(nodes.keys()))
# nodes

['Abandoned Ship',
 'Aqua Hideout',
 'Dewford',
 'Dive Hub',
 'E4 Pokecenter',
 'Fallarbor/Lavaridge',
 'Fortree/Lilycove',
 'Granite Cave',
 'Jagged Pass',
 'Lilycove Contest Hall',
 'Lilycove Department Store',
 'Magma Hideout',
 'Meteor Falls',
 'Mirage Tower',
 'Mossdeep',
 'Mt Pyre',
 'Navel Rock',
 'Oldale/Petalburg',
 'Pacifidlog',
 'Petalburg Woods',
 'Places of Interest',
 'Pokecenters',
 'Route 119/123',
 'Rustboro',
 'Rusturf Tunnel',
 'Seafloor Cavern',
 'Sky Pillar',
 'Slateport/Mauville/Verdanturf',
 'Sootopolis',
 'Surf Hub',
 'Victory Road',
 'Waterfall Hub']

In [34]:
sorted(list(edges.keys()))

['Abandoned Ship 1F Left Exit',
 'Abandoned Ship 1F Left N',
 'Abandoned Ship 1F Left S',
 'Abandoned Ship 1F Left Stairs',
 'Abandoned Ship 1F Right Exit',
 'Abandoned Ship 1F Right SW',
 'Abandoned Ship B1F SW',
 'Abandoned Ship Rooms NE',
 'Abandoned Ship Rooms SE',
 'Aqua Hideout B1F NE Room N',
 'Aqua Hideout B1F NE Room SE',
 'Aqua Hideout B1F NE Room SW',
 'Aqua Hideout B1F S Room E',
 'Aqua Hideout B1F S Room NW',
 'Aqua Hideout B1F S Room SW',
 'Aqua Hideout B2F NW',
 'Aqua Hideout B2F SW',
 'Aqua Hideout Warps 1 N',
 'Aqua Hideout Warps 1 S',
 'Aqua Hideout Warps 1 SE',
 'Aqua Hideout Warps 1 SW',
 'Aqua Hideout Warps 2 C',
 'Aqua Hideout Warps 2 E',
 'Aqua Hideout Warps 2 W',
 'Aqua Hideout Warps 3 C',
 'Aqua Hideout Warps 3 W',
 'Aqua Hideout Warps 4 C',
 'Aqua Hideout Warps 4 E',
 'Aqua Hideout Warps 4 W',
 'Contest Hall NW',
 'Contest Hall S',
 'Department Store 1F Exit',
 'Department Store 2F Up',
 'Department Store 3F Down',
 'Department Store 3F Up',
 'Department Store

In [54]:
[e['destination'] for e in nodes['Oldale/Petalburg']['transitions']]

['Aqua Hideout B1F S Room NW',
 'Slateport Museum',
 'Slateport SE',
 'Rusturf Tunnel S',
 'Pacifidlog NW',
 'Aqua Hideout B1F NE Room N']

In [52]:
nodes['Oldale/Petalburg']['transitions']

[{'entrance': 'Oldale Pokecenter',
  'destination': 'Aqua Hideout B1F S Room NW'},
 {'entrance': 'Oldale SE', 'destination': 'Slateport Museum'},
 {'entrance': 'Petalburg Gym', 'destination': 'Slateport SE'},
 {'entrance': 'Petalburg Mart', 'destination': 'Rusturf Tunnel S'},
 {'entrance': 'Petalburg SE', 'destination': 'Pacifidlog NW'},
 {'entrance': 'Petalburg SW', 'destination': 'Aqua Hideout B1F NE Room N'}]

In [52]:
lol = ['a', 'b', 'c']
lol.pop(0)
lol

['b', 'c']